In [213]:
import os
import glob
import numpy as np
import pandas as pd
import json
import datetime
import math

# importing required modules
from zipfile import ZipFile

In [2]:
_dir = 'D:/Flux_net/'

In [ ]:
BADM_file = _dir + 'FLX_AA-Flx_BIF_ALL_20200501/FLX_AA-Flx_BIF_HH_20200501.xlsx'

In [ ]:
badm = pd.read_excel(BADM_file)
badm

### Site info

In [ ]:
site_list  = list(badm['SITE_ID'].values)

In [ ]:
site_list = list(set(site_list))
site_list.sort()

In [ ]:
len(site_list)

In [ ]:
# site parameters

site_param = ['SITE_NAME','MAP','COUNTRY','IGBP', 'LOCATION_LAT', 'LOCATION_LONG','UTC_OFFSET' ]

In [ ]:
site_info = []
for _site in site_list:
    _data = dict()
    _data['site_id'] = _site
    for _param in site_param:
        _var = badm[(badm['SITE_ID'] == _site)][['VARIABLE','DATAVALUE']]
        _var_ = _var[_var['VARIABLE'] == _param]['DATAVALUE']
        if _var_.empty:
            _data[_param] = -9999
        else:
            _data[_param] = _var_.values[0]
    
    site_info.append(_data)

In [ ]:
pd_site_info = pd.DataFrame(site_info)
pd_site_info

In [ ]:
#pd_site_info.to_csv('site_info.csv', float_format='%.5f', index=False) 

### Site variables 

In [ ]:
_sel_param = ['TA_F','SW_IN_F','VPD_F','RH','WS_F','P','P_F',
             'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF', 'RECO_NT_VUT_REF','RECO_DT_VUT_REF','NEE_VUT_REF'
             ]

In [ ]:
site_vars = []
for _site in site_list:
    params = dict()
    params['site_id'] = _site
    _var = badm[(badm['SITE_ID'] == _site)][['VARIABLE','DATAVALUE']]

    block_start = 0

    for index, row in _var.iterrows():
        if row['VARIABLE'] == 'VAR_INFO_VARNAME':
            
            if block_start == 1:
                print('Err row',row)
            else:
                block_start = 1
                _param = row['DATAVALUE']

            continue
        

        if row['VARIABLE'] == 'VAR_INFO_UNIT':
            
            if block_start == 0:
                print('Err row',row)
            else:
                block_start = 0
                if _param not in _sel_param:
                    continue
                _unit = row['DATAVALUE']
                params[_param] = _unit
    
    site_vars.append(params)


In [ ]:
pd_site_param = pd.DataFrame(site_vars)
pd_site_param

In [ ]:
pd_site_param.iloc[136]

In [ ]:
pd_result = pd.concat([pd_site_info,pd_site_param], axis=1)
pd_result

In [ ]:
pd_site_param[pd_site_param['site_id'] == 'US-Wi0']

In [ ]:
pd_result.to_csv('all_sites_param.csv', float_format='%.5f', index=False)

### One site US-Wi0 flux net full HH data

In [ ]:
### flux net full data

US_Wi0_file = _dir + 'FLX_US-Wi0_FLUXNET2015_FULLSET_2002-2002_1-4/FLX_US-Wi0_FLUXNET2015_FULLSET_HH_2002-2002_1-4.csv'

data = pd.read_csv(US_Wi0_file, dtype={'TIMESTAMP_START':str, 'TIMESTAMP_END':str})
data

In [ ]:
def create_day(row):
    _str = row['TIMESTAMP_START']
    return _str[:-4]
data['day'] = data.apply (lambda row: create_day(row) , axis=1)

In [ ]:
data['SW_IN_F'].max()

In [ ]:
_sel_param = ['day','TA_F','SW_IN_F','VPD_F','RH','WS_F','P_F',
             'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF', 'RECO_NT_VUT_REF','RECO_DT_VUT_REF','NEE_VUT_REF'
             ] #'TIMESTAMP_START','TIMESTAMP_END','P',

In [ ]:
_qc_param = ['TA_F_QC','SW_IN_F_QC','VPD_F_QC','PA_F_QC','P_F_QC','WS_F_QC','NEE_VUT_REF_QC']

#### proce 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF', 'RECO_NT_VUT_REF','RECO_DT_VUT_REF','NEE_VUT_REF' daily average

In [ ]:
day_of_year = pd.DataFrame()
#day_of_year['day'] = _sel_pd['day'].unique()

In [ ]:
_sel_pd = data[['day','GPP_NT_VUT_REF', 'GPP_DT_VUT_REF', 'RECO_NT_VUT_REF','RECO_DT_VUT_REF','NEE_VUT_REF']]
_sel_pd_mean = _sel_pd.groupby(by = ['day']).mean().reset_index()

In [ ]:
_sel_pd_mean

In [ ]:
day_of_year = _sel_pd_mean.copy()
day_of_year

#### process TA_F daily max and min value from half hour data

In [ ]:
_sel_pd = data[['day','TA_F']]
_sel_taf_xx = _sel_pd.groupby(['day']).agg({'TA_F':['max','min']}).reset_index()
#_sel_pd_max = _sel_pd.groupby(by = ['day']).max().reset_index()
#_sel_pd_xx = _sel_pd.groupby(['day']).agg({'TA_F': ['mean', 'min', 'max'],'SW_IN_F': ['mean', 'min', 'max']}).reset_index()

In [ ]:
_sel_taf_xx.columns

In [ ]:
_sel_taf_xx[[('TA_F', 'max')]]

In [ ]:
day_of_year['TA_F_max'] = _sel_taf_xx[('TA_F', 'max')].copy()
day_of_year['TA_F_min'] = _sel_taf_xx[('TA_F', 'min')].copy()

In [ ]:
day_of_year

#### Process Humidity from RH, VPD_F and TA_F, then get per day's max and min humidity value

In [ ]:
def VPD2RH(Ta,VPD):
    #
    #ea = 0.611*np.exp(17.502*Td/(Td+240.97))
    es = 0.611*np.exp(17.502*Ta/(Ta+240.97))*10
    ea = es-VPD
    RH = ea*100/es
    RH[RH<0] = 0
    RH[RH>100] = 100    
    return RH

def ConvertRH2Humidty(Ta,RH,ALTI):
    #Ta, air temperature, C
    #Rh, relatively humidty, %
    #ALTI,altitude,m
    DWPTH=0.61*np.exp(5360.0*(3.661E-03-1.0/(273.15+Ta)))*max(0.0,min(100.0,RH))*0.01 #humidity
    TKA=Ta+273.15
    VPS=0.61*np.exp(5360.0*(3.661E-03-1.0/TKA))*np.exp(-ALTI/7272.0)
    VPK=min(DWPTH,VPS)
    return VPK

In [ ]:
data['VPD_F'].min()

In [ ]:
RH2= VPD2RH(data['TA_F'].values, data['VPD_F'].values)

In [ ]:
data['RH2'] = RH2

In [ ]:
data[['RH2']]

In [ ]:
RH2.mean(), RH2.max(),RH2.min()

In [ ]:
# calculate humidity
ALTI = 362  # This is a fixed value
def calculate_H(row):
    if row['RH'] < 0:
        humidity = ConvertRH2Humidty(row['TA_F'],row['RH2'],ALTI)
    else:
        humidity = ConvertRH2Humidty(row['TA_F'],row['RH'],ALTI)

    return humidity


data['Humidity'] = data.apply (lambda row: calculate_H(row) , axis=1)

In [ ]:
data[['Humidity']]

In [ ]:
_sel_pd = data[['day','Humidity']]
_sel_H_xx = _sel_pd.groupby(['day']).agg({'Humidity':['max','min']}).reset_index()

In [ ]:
day_of_year['H_max'] = _sel_H_xx[('Humidity', 'max')].copy()
day_of_year['H_min'] = _sel_H_xx[('Humidity', 'min')].copy()

In [ ]:
day_of_year

### Process SW_IN_F

In [ ]:
# RAND: SW_IN_F, J: hour,I:DOY
def ConvertSWDW(RADN,ALAT,ZNOON,J,I):
    #
    #
    TWILGT=0.06976
    #
    RADN = RADN*0.0036
    TYSIN = 0
    for N in range(1,5):
        YAZI=3.1416*(2*N-1)/4.0
        YAGL=3.1416/4.0
        YSIN=np.sin(YAGL)
        TYSIN=TYSIN+YSIN

    # Note below code are duplicate for DECDAY and below lines
    if ALAT > 0.0:
        XI=173
    else:
        XI=356
    DECDAY=XI+100
    DECLIN=np.sin((DECDAY*0.9863)*1.7453E-02)*(-23.47)
    AZI=np.sin(ALAT*1.7453E-02)*np.sin(DECLIN*1.7453E-02)
    DEC=np.cos(ALAT*1.7453E-02)*np.cos(DECLIN*1.7453E-02)
    
    XI=I
    if I==366:
        XI=365.5
    DECDAY=XI+100
    DECLIN=np.sin((DECDAY*0.9863)*1.7453E-02)*(-23.47)
    AZI=np.sin(ALAT*1.7453E-02)*np.sin(DECLIN*1.7453E-02)
    DEC=np.cos(ALAT*1.7453E-02)*np.cos(DECLIN*1.7453E-02)    
    
    #
    SSIN=max(0.0,AZI+DEC*np.cos(0.2618*(ZNOON-(J-0.5))))
    #
    if RADN <=0:
        SSIN = 0
    if SSIN <= TWILGT*-1:
        RADN = 0
        
    if SSIN > 0:
        RADX=4.896*max(0.0,SSIN)    
        RADN = min(RADX,RADN)
        #DIRECT VS DIFFUSE RADIATION IN SOLAR OR SKY BEAMS
        RADZ=min(RADN,0.5*(RADX-RADN))
        RADS=(RADN-RADZ)/SSIN
        RADS=min(4.167,RADS)    
        RADY=RADZ/TYSIN        
        
        TRAD = RADS*SSIN+RADY*TYSIN
    else:
        TRAD = 0
    
    return TRAD


In [ ]:
#RADN_hourly[year-start_year,DOY-1,hour-1] = ConvertSWDW(tmp_data['SW_IN_F'][i],ALAT,ZNOON,hour,DOY)

In [ ]:
pd_site_param =pd.read_csv('site_info.csv')
pd_site_param

In [ ]:
pd_site_param[pd_site_param['site_id'] == 'US-Wi0']['LOCATION_LAT'].values[0]

In [ ]:
ALAT = pd_site_param[pd_site_param['site_id'] == 'US-Wi0']['LOCATION_LAT'].values[0] #46.618778 # site_data.Latitude.values[sitei]
ZNOON = 12.5

In [ ]:
def calculate_RAND_HH(row):
    SW_IN_F = row['SW_IN_F']
    
    start_time = row['TIMESTAMP_START']
    year = int(start_time[:4])
    month = int(start_time[4:6])
    day_of_month = int(start_time[6:8])
    hour = int(start_time[8:10]) + 1   # hour is from 1 - 24
    day_of_year = datetime.datetime(int(year), int(month), int(day_of_month)).timetuple().tm_yday
    return ConvertSWDW(SW_IN_F,ALAT,ZNOON,hour,day_of_year)

In [ ]:
data['RAND_HH'] = data.apply (lambda row: calculate_RAND_HH(row) , axis=1)

In [ ]:
data['RAND_HH'].describe()

In [ ]:
# RADN = np.sum(RADN_hourly,axis=-1)

_sel_pd = data[['day','RAND_HH']]
_sel_H_xx = _sel_pd.groupby(['day']).agg({'RAND_HH':'sum'}).reset_index()

In [ ]:
day_of_year['RAND'] = _sel_H_xx['RAND_HH']

### Process wind speed

In [ ]:
def ConvertWindSpeed(row):
    #WS, wind speed, m/s
    WS = row['WS_F']
    WINDH=WS*3600.0 #m/h
    UA = max(3600,WINDH)*0.001 #km/h
    return UA

In [ ]:
data['WIND_HH'] = data.apply (lambda row: ConvertWindSpeed(row) , axis=1)

In [ ]:
_sel_pd = data[['day','WIND_HH']]
_sel_H_xx = _sel_pd.groupby(['day']).agg({'WIND_HH':'sum'}).reset_index()
day_of_year['WIND'] = _sel_H_xx['WIND_HH']

### Process PRECN

In [ ]:
_sel_pd = data[['day','P_F']]
_sel_H_xx = _sel_pd.groupby(['day']).agg({'P_F':'sum'}).reset_index()
day_of_year['PRECN'] = _sel_H_xx['P_F']

In [ ]:
day_of_year

In [ ]:
day_of_year.to_csv('processed_day_values.csv', float_format='%.5f', index=False) 

## QC data

In [ ]:
US_Wi0_day_file = _dir + 'FLX_US-Wi0_FLUXNET2015_FULLSET_2002-2002_1-4/FLX_US-Wi0_FLUXNET2015_FULLSET_DD_2002-2002_1-4.csv'

day_data = pd.read_csv(US_Wi0_day_file, dtype={'TIMESTAMP':str})

In [ ]:
day_data

In [ ]:
_qc_param = ['TIMESTAMP','TA_F_QC','SW_IN_F_QC','VPD_F_QC','PA_F_QC','P_F_QC','WS_F_QC','NEE_VUT_REF_QC']
QC = day_data[_qc_param]
QC

In [ ]:
QC.to_csv('DAY_QC_values.csv', float_format='%.5f', index=False) 

# Process all sites fullnet data

In [20]:
_root = 'D:/Flux_net/'
all_zip = glob.glob(_root + "*FLUXNET2015_FULLSET*.zip")
len(all_zip)

206

In [21]:
all_zip

['D:/Flux_net\\FLX_AR-SLu_FLUXNET2015_FULLSET_2009-2011_1-4.zip',
 'D:/Flux_net\\FLX_AR-Vir_FLUXNET2015_FULLSET_2009-2012_1-4.zip',
 'D:/Flux_net\\FLX_AT-Neu_FLUXNET2015_FULLSET_2002-2012_1-4.zip',
 'D:/Flux_net\\FLX_AU-Ade_FLUXNET2015_FULLSET_2007-2009_1-4.zip',
 'D:/Flux_net\\FLX_AU-ASM_FLUXNET2015_FULLSET_2010-2014_2-4.zip',
 'D:/Flux_net\\FLX_AU-Cpr_FLUXNET2015_FULLSET_2010-2014_2-4.zip',
 'D:/Flux_net\\FLX_AU-Cum_FLUXNET2015_FULLSET_2012-2014_2-4.zip',
 'D:/Flux_net\\FLX_AU-DaP_FLUXNET2015_FULLSET_2007-2013_2-4.zip',
 'D:/Flux_net\\FLX_AU-DaS_FLUXNET2015_FULLSET_2008-2014_2-4.zip',
 'D:/Flux_net\\FLX_AU-Dry_FLUXNET2015_FULLSET_2008-2014_2-4.zip',
 'D:/Flux_net\\FLX_AU-Emr_FLUXNET2015_FULLSET_2011-2013_1-4.zip',
 'D:/Flux_net\\FLX_AU-Fog_FLUXNET2015_FULLSET_2006-2008_1-4.zip',
 'D:/Flux_net\\FLX_AU-Gin_FLUXNET2015_FULLSET_2011-2014_1-4.zip',
 'D:/Flux_net\\FLX_AU-GWW_FLUXNET2015_FULLSET_2013-2014_1-4.zip',
 'D:/Flux_net\\FLX_AU-How_FLUXNET2015_FULLSET_2001-2014_1-4.zip',
 'D:/Flux_

In [ ]:
file_name = all_zip[6]

file_prefix = os.path.basename(file_name)[:-4]

In [ ]:
file_prefix

In [ ]:
file_prefix.split('_')

In [ ]:
'FLX_AU-Cum_FLUXNET2015_FULLSET_HH_2012-2014_2-4.csv'

In [ ]:
len('_2012-2014_2-4')

In [ ]:
file_prefix[:-14], file_prefix[-14:]

In [ ]:
hh_file = file_prefix[:-14]+ '_HH'+ file_prefix[-14:]
hh_file

In [ ]:
_sel_param = ['TA_F','SW_IN_F','VPD_F','RH','WS_F','P_F',
             'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF', 'RECO_NT_VUT_REF','RECO_DT_VUT_REF','NEE_VUT_REF'
             ]

In [ ]:
# specifying the zip file name
for file_name in all_zip:
    
    # opening the zip file in READ mode
    with ZipFile(file_name, 'r') as zip:
        # printing all the contents of the zip file
        #zip.printdir()
        file_list = zip.namelist()
        
        file_prefix = os.path.basename(file_name)[:-4]
        
        hh_file = file_prefix[:-14]+ '_HH'+ file_prefix[-14:] + '.csv'
        hr_file = file_prefix[:-14]+ '_HR'+ file_prefix[-14:] + '.csv'
        dd_file = file_prefix[:-14]+ '_DD'+ file_prefix[-14:] + '.csv'
        
        #dd_data = zip.read(dd_file)
        
        #pd_data = pd.read_csv(zip.open(dd_file), dtype={'TIMESTAMP':str})
        hour_file = ''
        if hh_file in file_list :
            hour_file = hh_file
        elif hr_file in file_list :
            hour_file = hr_file

        if hour_file != '':
            data = pd.read_csv(zip.open(hour_file), dtype={'TIMESTAMP_START':str, 'TIMESTAMP_END':str})
            # check if necessary parameters exist in Hourly file
            for item in _sel_param :
                if item not in data.columns:
                    print(hour_file, item)
            #process_hour_file(data, site_id, pd_site_param, output_dir)


print('Done!')

In [ ]:
pd_data

### Get each site start year and end year for site info

In [23]:
flux_site = pd.read_csv('all_sites_param.csv')

In [41]:
flux_site.insert(2,'end year',0)
flux_site.insert(2,'start year',0)

In [42]:
flux_site

,site_id,SITE_NAME,start year,end year,MAP,COUNTRY,IGBP,LOCATION_LAT,LOCATION_LONG,UTC_OFFSET,...,VPD_F,P,P_F,WS_F,RH,NEE_VUT_REF,RECO_NT_VUT_REF,GPP_NT_VUT_REF,RECO_DT_VUT_REF,GPP_DT_VUT_REF
0,AR-SLu,San Luis,0,0,400.0,Argentina,MF,-33.464800,-66.459800,-3.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
1,AR-Vir,Virasoro,0,0,-9999.0,Argentina,ENF,-28.239500,-56.188600,-3.0,...,hPa,NaN,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
2,AT-Neu,Neustift,0,0,852.0,Austria,GRA,47.116670,11.317500,1.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
3,AU-ASM,Alice Springs,0,0,-9999.0,Australia,SAV,-22.283000,133.249000,9.5,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
4,AU-Ade,Adelaide River,0,0,-9999.0,Australia,WSA,-13.076900,131.117800,9.5,...,hPa,NaN,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,US-Wi8,Young hardwood clearcut (YHW),0,0,-9999.0,USA,DBF,46.722333,-91.252417,-6.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
208,US-Wi9,Young Jack pine (YJP),0,0,-9999.0,USA,ENF,46.618778,-91.081444,-6.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
209,US-Wkg,Walnut Gulch Kendall Grasslands,0,0,407.0,USA,GRA,31.736500,-109.941900,-7.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
210,ZA-Kru,Skukuza,0,0,547.0,South Africa,SAV,-25.019700,31.496900,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
for file_name in all_zip:
    # opening the zip file in READ mode
    with ZipFile(file_name, 'r') as zip:
        # printing all the contents of the zip file
        #zip.printdir()
        file_list = zip.namelist()
        
        file_prefix = os.path.basename(file_name)[:-4]
        site_id = file_prefix.split('_')[1]
        
        yy_file = file_prefix[:-14]+ '_YY'+ file_prefix[-14:] + '.csv'
        if yy_file in file_list:
            pd_data = pd.read_csv(zip.open(yy_file), dtype={'TIMESTAMP':str})
            flux_site.loc[flux_site['site_id'] == site_id, 'start year'] = pd_data['TIMESTAMP'].values[0]
            flux_site.loc[flux_site['site_id'] == site_id, 'end year'] = pd_data['TIMESTAMP'].values[-1]
        


In [51]:
flux_site

,site_id,SITE_NAME,start year,end year,MAP,COUNTRY,IGBP,LOCATION_LAT,LOCATION_LONG,UTC_OFFSET,...,VPD_F,P,P_F,WS_F,RH,NEE_VUT_REF,RECO_NT_VUT_REF,GPP_NT_VUT_REF,RECO_DT_VUT_REF,GPP_DT_VUT_REF
0,AR-SLu,San Luis,2009,2011,400.0,Argentina,MF,-33.464800,-66.459800,-3.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
1,AR-Vir,Virasoro,2009,2012,-9999.0,Argentina,ENF,-28.239500,-56.188600,-3.0,...,hPa,NaN,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
2,AT-Neu,Neustift,2002,2012,852.0,Austria,GRA,47.116670,11.317500,1.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
3,AU-ASM,Alice Springs,2010,2014,-9999.0,Australia,SAV,-22.283000,133.249000,9.5,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
4,AU-Ade,Adelaide River,2007,2009,-9999.0,Australia,WSA,-13.076900,131.117800,9.5,...,hPa,NaN,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,US-Wi8,Young hardwood clearcut (YHW),2002,2002,-9999.0,USA,DBF,46.722333,-91.252417,-6.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
208,US-Wi9,Young Jack pine (YJP),2004,2005,-9999.0,USA,ENF,46.618778,-91.081444,-6.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
209,US-Wkg,Walnut Gulch Kendall Grasslands,2004,2014,407.0,USA,GRA,31.736500,-109.941900,-7.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
210,ZA-Kru,Skukuza,0,0,547.0,South Africa,SAV,-25.019700,31.496900,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Process SRDB data

In [ ]:
#dtype={'Study_midyear':float, 'YearsOfData':float,'Collar_height':float, 'RC_season':str}

In [231]:
srdb_data = pd.read_csv('srdb-data-V5.csv',dtype={'Study_midyear':float, 'YearsOfData':float,'Duplicate_record':str,
                                                  'Collar_height':str, 'RC_season':str})
srdb_data

,Record_number,Entry_date,Study_number,Author,Duplicate_record,Quality_flag,Contributor,Country,Region,Site_name,...,BA,C_veg_total,C_AG,C_BG,C_CR,C_FR,C_litter,C_soilmineral,C_soildepth,Notes
0,1.0,2008-09-26,2534,Bond-Lamberty,NaN,Q02,BBL,Canada,Manitoba,BOREAS NSA-D1998,...,0.0,200.0,80.0,120.0,70.0,50.0,NaN,41800.0,70.0,LAI from Bond-Lamberty (2002); C from Wang (20...
1,2.0,2008-09-26,2534,Bond-Lamberty,NaN,Q02,BBL,Canada,Manitoba,BOREAS NSA-D1995,...,0.5,160.0,90.0,70.0,40.0,30.0,NaN,41800.0,70.0,LAI from Bond-Lamberty (2002); C from Wang (20...
2,3.0,2008-09-26,2534,Bond-Lamberty,NaN,Q02,BBL,Canada,Manitoba,BOREAS NSA-D1989,...,0.0,520.0,400.0,120.0,70.0,50.0,NaN,41800.0,70.0,LAI from Bond-Lamberty (2002); C from Wang (20...
3,4.0,2008-09-26,2534,Bond-Lamberty,NaN,Q02,BBL,Canada,Manitoba,BOREAS NSA-D1981,...,4.1,1050.0,700.0,350.0,230.0,120.0,NaN,41800.0,70.0,LAI from Bond-Lamberty (2002); C from Wang (20...
4,5.0,2008-09-26,2534,Bond-Lamberty,NaN,Q02,BBL,Canada,Manitoba,BOREAS NSA-D1964,...,11.6,2300.0,1500.0,800.0,600.0,200.0,NaN,41800.0,70.0,LAI from Bond-Lamberty (2002); C from Wang (20...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10361,10425.0,2020-04-13,11965,Ivanov,NaN,NaN,NK,Russia,Far East,Primorskii,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10362,10426.0,2020-04-13,11965,Ivanov,NaN,NaN,NK,Russia,Far East,Primorskii,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10363,10427.0,2020-04-13,11966,Ivanov,NaN,NaN,NK,Russia,Far East,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10364,10428.0,2020-04-14,12000,Yuzbekov,NaN,NaN,NK,Russia,Novgorod,Valday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
srdb_data.columns[[4,39,61]]

Index(['Duplicate_record', 'Collar_height', 'RC_season'], dtype='object')

#### Create SRDB related start year and end year in a new Data Frame

In [232]:
srdb_years = srdb_data[['Record_number','Site_name','Site_ID','Latitude','Longitude','Study_midyear','YearsOfData']].copy()
srdb_years

,Record_number,Site_name,Site_ID,Latitude,Longitude,Study_midyear,YearsOfData
0,1.0,BOREAS NSA-D1998,CA-BND-3YRWD,56.630,-99.940,2001.5,1.5
1,2.0,BOREAS NSA-D1995,CA-BND-6YRWD,56.460,-99.970,2001.5,1.5
2,3.0,BOREAS NSA-D1989,CA-BND-12YRWD,55.910,-98.980,2001.5,1.5
3,4.0,BOREAS NSA-D1981,CA-BND-20YRWD,55.860,-98.480,2001.5,1.5
4,5.0,BOREAS NSA-D1964,CA-BND-37YRWD,55.920,-98.390,2001.5,1.5
...,...,...,...,...,...,...,...
10361,10425.0,Primorskii,RU-FE-PRIM,43.630,132.250,2015.0,1.0
10362,10426.0,Primorskii,RU-FE-PRIM,43.630,132.250,2016.0,1.0
10363,10427.0,NaN,RU-FE,46.630,132.260,2015.0,1.0
10364,10428.0,Valday,RU-NOVG-MG,50.012,33.345,NaN,1.0


In [233]:
def get_startyear(row):
    if pd.isnull(row['Study_midyear']) or pd.isnull(row['YearsOfData']):
        return ''
    tmp = row['Study_midyear'] - row['YearsOfData']*0.5
    remain,year = math.modf(tmp)
    return str(int(year))+'.'+str(int(remain*12))
    

In [234]:
srdb_years['start_year'] = srdb_data.apply(lambda row: get_startyear(row),axis=1)

In [235]:
def get_endyear(row):
    if pd.isnull(row['Study_midyear']) or pd.isnull(row['YearsOfData']):
        return ''
    tmp = row['Study_midyear'] + row['YearsOfData']*0.5
    remain,year = math.modf(tmp)
    return str(int(year))+'.'+str(int(remain*12))

In [236]:
srdb_years['end_year'] = srdb_data.apply(lambda row: get_endyear(row),axis=1)

In [237]:
srdb_years

,Record_number,Site_name,Site_ID,Latitude,Longitude,Study_midyear,YearsOfData,start_year,end_year
0,1.0,BOREAS NSA-D1998,CA-BND-3YRWD,56.630,-99.940,2001.5,1.5,2000.9,2002.3
1,2.0,BOREAS NSA-D1995,CA-BND-6YRWD,56.460,-99.970,2001.5,1.5,2000.9,2002.3
2,3.0,BOREAS NSA-D1989,CA-BND-12YRWD,55.910,-98.980,2001.5,1.5,2000.9,2002.3
3,4.0,BOREAS NSA-D1981,CA-BND-20YRWD,55.860,-98.480,2001.5,1.5,2000.9,2002.3
4,5.0,BOREAS NSA-D1964,CA-BND-37YRWD,55.920,-98.390,2001.5,1.5,2000.9,2002.3
...,...,...,...,...,...,...,...,...,...
10361,10425.0,Primorskii,RU-FE-PRIM,43.630,132.250,2015.0,1.0,2014.6,2015.6
10362,10426.0,Primorskii,RU-FE-PRIM,43.630,132.250,2016.0,1.0,2015.6,2016.6
10363,10427.0,NaN,RU-FE,46.630,132.260,2015.0,1.0,2014.6,2015.6
10364,10428.0,Valday,RU-NOVG-MG,50.012,33.345,NaN,1.0,,


In [238]:
flux_site = pd.read_csv('all_sites_param.csv')
flux_site_loc = flux_site[['site_id','LOCATION_LAT','LOCATION_LONG']].copy()
flux_site_loc

,site_id,LOCATION_LAT,LOCATION_LONG
0,AR-SLu,-33.46480,-66.45980
1,AR-Vir,-28.23950,-56.18860
2,AT-Neu,47.11667,11.31750
3,AU-ASM,-22.28300,133.24900
4,AU-Ade,-13.07690,131.11780
...,...,...,...
207,US-Wi8,46.72233,-91.25242
208,US-Wi9,46.61878,-91.08144
209,US-Wkg,31.73650,-109.94190
210,ZA-Kru,-25.01970,31.49690


In [202]:
_pd.to_csv()

,site_id,LOCATION_LAT,LOCATION_LONG,index,Record_number,Entry_date,Study_number,Author,Duplicate_record,Quality_flag,...,BA,C_veg_total,C_AG,C_BG,C_CR,C_FR,C_litter,C_soilmineral,C_soildepth,Notes
0,AU-Tum,-35.6566,148.1517,3671,3707.0,2010-05-11,4920,Keith,NaN,NaN,...,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AU-Tum,-35.6566,148.1517,3672,3708.0,2010-05-11,4920,Keith,NaN,NaN,...,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AU-Tum,-35.6566,148.1517,3673,3709.0,2010-05-11,4920,Keith,NaN,NaN,...,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [242]:
#Write to files, one site one file

_dir = './site_SRDB/group1/'
gap = 0.01

output_dir = _dir + 'gap_'+ str(gap) + '/'


In [243]:
output_dir

'./site_SRDB/group1/gap_0.01/'

In [105]:
_match_long

,Record_number,Site_name,Study_midyear,YearsOfData,Latitude,Longitude,MAT,MAP,Rs_annual,Ra_annual,...,Rs_summer,Rs_autumn,Rs_winter,Rs_growingseason,GPP,ER,NEP,NPP,ANPP,BNPP
7845,7900.0,NaN,2010.5,1.0,-28.23,-56.19,21.1,1800.0,NaN,NaN,...,NaN,NaN,NaN,NaN,4290.0,NaN,NaN,NaN,NaN,NaN
7846,7901.0,NaN,2011.5,1.0,-28.23,-56.19,21.1,1800.0,NaN,NaN,...,NaN,NaN,NaN,NaN,4015.0,NaN,NaN,NaN,NaN,NaN


In [244]:
# Group 1, fluxnet sites can related to SRDB
group_1 = list()
srdb_group1 = list()
for _idx, site_row in flux_site_loc.iterrows():
    lat  = site_row['LOCATION_LAT']
    long = site_row['LOCATION_LONG']

    _match_lat = srdb_data[abs(srdb_data['Latitude'] - lat) <= gap]

    if _match_lat.shape[0] > 0:
        #for _idx, srdb_row in _match_lat.iterrows():
        _match_long = _match_lat[abs(_match_lat['Longitude'] - long) <= gap]

        if _match_long.shape[0] > 0:
            site_id = site_row['site_id']
            file_name = output_dir + site_id + '_SRDB.csv'
            srdb_years_name = output_dir + site_id + '_SRDB_years.csv'
            
            group_1.append(site_id)
            srdb_group1.extend(_match_long.index.tolist())
            
            row_years = srdb_years.iloc[_match_long.index]
            _match_long.to_csv(file_name, float_format='%.5f', index=False)
            print('output file: ', file_name)
            
            row_years.to_csv(srdb_years_name,float_format='%.5f', index=False)
            print('output file: ', srdb_years_name)
            
    

output file:  ./site_SRDB/group1/gap_0.01/AR-Vir_SRDB.csv
output file:  ./site_SRDB/group1/gap_0.01/AR-Vir_SRDB_years.csv
output file:  ./site_SRDB/group1/gap_0.01/AT-Neu_SRDB.csv
output file:  ./site_SRDB/group1/gap_0.01/AT-Neu_SRDB_years.csv
output file:  ./site_SRDB/group1/gap_0.01/AU-Tum_SRDB.csv
output file:  ./site_SRDB/group1/gap_0.01/AU-Tum_SRDB_years.csv
output file:  ./site_SRDB/group1/gap_0.01/AU-Wom_SRDB.csv
output file:  ./site_SRDB/group1/gap_0.01/AU-Wom_SRDB_years.csv
output file:  ./site_SRDB/group1/gap_0.01/BE-Bra_SRDB.csv
output file:  ./site_SRDB/group1/gap_0.01/BE-Bra_SRDB_years.csv
output file:  ./site_SRDB/group1/gap_0.01/BE-Lon_SRDB.csv
output file:  ./site_SRDB/group1/gap_0.01/BE-Lon_SRDB_years.csv
output file:  ./site_SRDB/group1/gap_0.01/BE-Vie_SRDB.csv
output file:  ./site_SRDB/group1/gap_0.01/BE-Vie_SRDB_years.csv
output file:  ./site_SRDB/group1/gap_0.01/CA-Gro_SRDB.csv
output file:  ./site_SRDB/group1/gap_0.01/CA-Gro_SRDB_years.csv
output file:  ./site_SRD

In [245]:
row_years = srdb_years.iloc[_match_long.index]
row_years

,Record_number,Site_name,Site_ID,Latitude,Longitude,Study_midyear,YearsOfData,start_year,end_year
4373,4407.0,Kataba Forest Reserve,ZM-KFR-PLOT1,-15.43,23.25,2008.0,1.0,2007.6,2008.6
4374,4408.0,Kataba Forest Reserve,ZM-KFR-PLOT2,-15.43,23.25,2008.0,1.0,2007.6,2008.6
4375,4409.0,Kataba Forest Reserve,ZM-KFR-PLOT3,-15.43,23.25,2008.0,1.0,2007.6,2008.6
4376,4410.0,Kataba Forest Reserve,ZM-KFR-PLOT1,-15.43,23.25,2009.0,1.0,2008.6,2009.6
4377,4411.0,Kataba Forest Reserve,ZM-KFR-PLOT2,-15.43,23.25,2009.0,1.0,2008.6,2009.6
4378,4412.0,Kataba Forest Reserve,ZM-KFR-PLOT3,-15.43,23.25,2009.0,1.0,2008.6,2009.6
4379,4413.0,Kataba Forest Reserve,ZM-KFR-PLOT4,-15.43,23.25,2009.0,1.0,2008.6,2009.6


In [246]:
_match_long

,Record_number,Entry_date,Study_number,Author,Duplicate_record,Quality_flag,Contributor,Country,Region,Site_name,...,BA,C_veg_total,C_AG,C_BG,C_CR,C_FR,C_litter,C_soilmineral,C_soildepth,Notes
4373,4407.0,2012-03-11,6151,Merbold,NaN,NaN,BBL,Zambia,Western Province,Kataba Forest Reserve,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4374,4408.0,2012-03-11,6151,Merbold,NaN,NaN,BBL,Zambia,Western Province,Kataba Forest Reserve,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4375,4409.0,2012-03-11,6151,Merbold,NaN,NaN,BBL,Zambia,Western Province,Kataba Forest Reserve,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4376,4410.0,2012-03-11,6151,Merbold,NaN,NaN,BBL,Zambia,Western Province,Kataba Forest Reserve,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4377,4411.0,2012-03-11,6151,Merbold,NaN,NaN,BBL,Zambia,Western Province,Kataba Forest Reserve,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4378,4412.0,2012-03-11,6151,Merbold,NaN,NaN,BBL,Zambia,Western Province,Kataba Forest Reserve,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4379,4413.0,2012-03-11,6151,Merbold,NaN,NaN,BBL,Zambia,Western Province,Kataba Forest Reserve,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [247]:
len(group_1)

82

In [248]:
srdb_group1

[7845,
 7846,
 2311,
 9479,
 3671,
 3672,
 3673,
 6012,
 6013,
 8713,
 8714,
 8715,
 1041,
 1042,
 1043,
 1044,
 1045,
 1046,
 1945,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2062,
 2322,
 2323,
 2324,
 2325,
 2326,
 2327,
 2328,
 2329,
 2330,
 2331,
 2332,
 2333,
 2662,
 4425,
 4519,
 560,
 561,
 2057,
 2064,
 136,
 137,
 6,
 13,
 20,
 27,
 301,
 302,
 303,
 1443,
 1773,
 6,
 13,
 20,
 27,
 301,
 302,
 303,
 1443,
 1773,
 5,
 19,
 4,
 11,
 18,
 25,
 4,
 11,
 18,
 25,
 3,
 10,
 17,
 24,
 0,
 7,
 14,
 21,
 121,
 164,
 41,
 42,
 78,
 79,
 89,
 122,
 451,
 2975,
 3769,
 4965,
 1401,
 1400,
 5857,
 5858,
 4255,
 5859,
 5860,
 4255,
 5859,
 5860,
 4721,
 4722,
 7322,
 8004,
 8005,
 5639,
 3868,
 3869,
 3870,
 3871,
 3872,
 3873,
 3874,
 3875,
 3876,
 3877,
 3878,
 3879,
 3880,
 3881,
 3882,
 3883,
 3884,
 3885,
 3886,
 3887,
 3888,
 3889,
 3890,
 3891,
 3892,
 3893,
 3894,
 3895,
 3896,
 3897,
 3898,
 3899,
 3900,
 3901,
 3902,
 3903,
 4200,
 4201,
 4202,
 4203,
 4204,
 4205,
 4206,
 4207,

In [85]:
group_2 = flux_site.copy()
drop_idx = []
for idx, row in group_2.iterrows():
    if row['site_id'] in group_1:
        drop_idx.append(idx)

In [86]:
group_2 = group_2.drop(drop_idx)

In [84]:
group_2 = group_2.drop(drop_idx).reset_index()
group_2 = group_2.drop(['index'],axis=1)
group_2

,site_id,SITE_NAME,start year,end year,MAP,COUNTRY,IGBP,LOCATION_LAT,LOCATION_LONG,UTC_OFFSET,...,VPD_F,P,P_F,WS_F,RH,NEE_VUT_REF,RECO_NT_VUT_REF,GPP_NT_VUT_REF,RECO_DT_VUT_REF,GPP_DT_VUT_REF
0,AR-SLu,San Luis,2009,2011,400.0,Argentina,MF,-33.464800,-66.459800,-3.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
1,AU-ASM,Alice Springs,2010,2014,-9999.0,Australia,SAV,-22.283000,133.249000,9.5,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
2,AU-Ade,Adelaide River,2007,2009,-9999.0,Australia,WSA,-13.076900,131.117800,9.5,...,hPa,NaN,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
3,AU-Cpr,Calperum,2010,2014,-9999.0,Australia,SAV,-34.002100,140.589100,9.5,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
4,AU-Cum,Cumberland Plain,2012,2014,-9999.0,Australia,EBF,-33.615180,150.723620,10.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,US-Wi6,Pine barrens #1 (PB1),2002,2003,-9999.0,USA,OSH,46.624889,-91.298222,-6.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
126,US-Wi7,Red pine clearcut (RPCC),2005,2005,-9999.0,USA,OSH,46.649111,-91.069278,-6.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
127,US-Wi8,Young hardwood clearcut (YHW),2002,2002,-9999.0,USA,DBF,46.722333,-91.252417,-6.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1
128,US-Wi9,Young Jack pine (YJP),2004,2005,-9999.0,USA,ENF,46.618778,-91.081444,-6.0,...,hPa,mm,mm,m s-1,%,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1,µmolCO2 m-2 s-1


In [88]:
group_2.to_csv('group_2_site_info.csv',float_format='%.5f', index=False)

In [ ]:
# Group 3, SRDB data except related to Flux sites

In [250]:
group3 = srdb_data.copy()

In [251]:
group3 = group3.drop(srdb_group1)
group3

,Record_number,Entry_date,Study_number,Author,Duplicate_record,Quality_flag,Contributor,Country,Region,Site_name,...,BA,C_veg_total,C_AG,C_BG,C_CR,C_FR,C_litter,C_soilmineral,C_soildepth,Notes
1,2.0,2008-09-26,2534,Bond-Lamberty,NaN,Q02,BBL,Canada,Manitoba,BOREAS NSA-D1995,...,0.5,160.0,90.0,70.0,40.0,30.0,NaN,41800.0,70.0,LAI from Bond-Lamberty (2002); C from Wang (20...
2,3.0,2008-09-26,2534,Bond-Lamberty,NaN,Q02,BBL,Canada,Manitoba,BOREAS NSA-D1989,...,0.0,520.0,400.0,120.0,70.0,50.0,NaN,41800.0,70.0,LAI from Bond-Lamberty (2002); C from Wang (20...
8,9.0,2008-09-26,2534,Bond-Lamberty,NaN,Q02,BBL,Canada,Manitoba,BOREAS NSA-W1995,...,0.0,550.0,100.0,450.0,300.0,150.0,NaN,NaN,NaN,LAI from Bond-Lamberty (2002); C from Wang (20...
9,10.0,2008-09-26,2534,Bond-Lamberty,NaN,Q02,BBL,Canada,Manitoba,BOREAS NSA-W1989,...,0.0,740.0,350.0,390.0,260.0,130.0,NaN,NaN,NaN,LAI from Bond-Lamberty (2002); C from Wang (20...
12,13.0,2008-09-26,2534,Bond-Lamberty,NaN,Q02,BBL,Canada,Manitoba,BOREAS NSA-W1930,...,20.3,3880.0,3400.0,480.0,340.0,140.0,NaN,NaN,NaN,LAI from Bond-Lamberty (2002); C from Wang (20...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10361,10425.0,2020-04-13,11965,Ivanov,NaN,NaN,NK,Russia,Far East,Primorskii,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10362,10426.0,2020-04-13,11965,Ivanov,NaN,NaN,NK,Russia,Far East,Primorskii,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10363,10427.0,2020-04-13,11966,Ivanov,NaN,NaN,NK,Russia,Far East,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10364,10428.0,2020-04-14,12000,Yuzbekov,NaN,NaN,NK,Russia,Novgorod,Valday,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [252]:
group3.to_csv('group3_SRDB.csv',float_format='%.5f', index=False)

In [ ]:
row